In [1]:
from spamEmail import SpamEmailBayes
import re

In [2]:
spam = SpamEmailBayes()

In [3]:
spamDict = {}
normDict = {}
testDict = {}

In [4]:
wordsList = []
wordsDict = {}

In [5]:
testResult = {}

In [6]:
normFileList = spam.get_file_list('../data/normal/')
spamFileList = spam.get_file_list('../data/spam')
testFileList = spam.get_file_list('../data/test')

In [7]:
normFileLen = len(normFileList)
spamFileLen = len(spamFileList)

In [8]:
stop_list = spam.get_stop_words()

In [9]:
for file_name in normFileList:
    wordsList.clear()
    with open('../data/normal/' + file_name, 'r', encoding='gbk') as f:
        content = f.read()
        for line in content:
            rule = re.compile(u'[^\u4E00-\u9FA5]')
            line = rule.sub(r'', line)
            spam.get_word_list(line, wordsList, stop_list)
    
    spam.addToDict(wordsList, wordsDict)

normDict = wordsDict.copy()

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/jc/l9vx9tp979g0tm976wjrgwkr0000gn/T/jieba.cache
Loading model cost 1.017 seconds.
Prefix dict has been built succesfully.


In [10]:
for file_name in spamFileList:
    wordsList.clear()
    with open('../data/spam/' + file_name, 'r', encoding='gbk') as f:
        content = f.read()
        for line in content:
            rule = re.compile(u'[^\u4E00-\u9FA5]')
            line = rule.sub(r'', line)
            spam.get_word_list(line, wordsList, stop_list)
    
    spam.addToDict(wordsList, wordsDict)

spamDict = wordsDict.copy()

In [11]:
for file_name in testFileList:
    testDict.clear()
    wordsDict.clear()
    wordsList.clear()
    with open('../data/test/' + file_name, 'r', encoding='gbk') as f:
        content = f.read()
        for line in content:
            rule = re.compile(u'[^\u4E00-\u9FA5]')
            line = rule.sub(r'', line)
            spam.get_word_list(line, wordsList, stop_list)
    
    spam.addToDict(wordsList, wordsDict)
    testDict = wordsDict.copy()
    
    word_prob_list = spam.get_test_words(testDict, spamDict, normDict, normFileLen, spamFileLen)
  
    p = spam.calBayes(word_prob_list)
    
    if (p > 0.9):
        testResult.setdefault(file_name, 1)
    else:
        testResult.setdefault(file_name, 0)

In [12]:
testAccuracy = spam.calAccuracy(testResult)
# for file, cat in testResult.items():
#     print(file + '/' + str(cat))
print(testAccuracy)

0.5076530612244898
